In [1]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 KB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.8/112.8 KB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 12.7 MB/s eta 0:00:00
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting un

In [2]:
pip install lazypredict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import date
from dateutil.relativedelta import relativedelta
import yfinance as yf
from datetime import datetime
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
from sklearn.model_selection import train_test_split

In [4]:
start_date = date.today()  - relativedelta(years=5)
historic_data = yf.download('^NSEI',start_date,date.today())
historic_data.index = historic_data.index.strftime('%Y-%m-%d')
historic_data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-03-12,10301.599609,10433.650391,10295.450195,10421.400391,10421.400391,216400
2018-03-13,10389.500000,10478.599609,10377.849609,10426.849609,10426.849609,266700
2018-03-14,10393.049805,10420.349609,10336.299805,10410.900391,10410.900391,181400
2018-03-15,10405.450195,10420.000000,10346.200195,10360.150391,10360.150391,173900
2018-03-16,10345.150391,10346.299805,10180.250000,10195.150391,10195.150391,328500
...,...,...,...,...,...,...
2023-03-03,17451.250000,17644.750000,17427.699219,17594.349609,17594.349609,356200
2023-03-06,17680.349609,17799.949219,17671.949219,17711.449219,17711.449219,362800
2023-03-08,17665.750000,17766.500000,17602.250000,17754.400391,17754.400391,267000


In [7]:
sentiment_data = pd.read_csv('/content/drive/MyDrive/StockPricePrediction/CreateSentimentDataset/csv_files/final_sentiment.csv', usecols=["Date Created", "Mode"])
sentiment_data.columns = ['Date','Sentiment']
sentiment_data.set_index('Date',inplace=True)

In [8]:
data = pd.merge(historic_data, sentiment_data, left_index=True, right_index=True)
data.dropna(inplace=True)
data

,Open,High,Low,Close,Adj Close,Volume,Sentiment
Date,,,,,,,
2018-03-12,10301.599609,10433.650391,10295.450195,10421.400391,10421.400391,216400,-1
2018-03-13,10389.500000,10478.599609,10377.849609,10426.849609,10426.849609,266700,-1
2018-03-14,10393.049805,10420.349609,10336.299805,10410.900391,10410.900391,181400,1
2018-03-15,10405.450195,10420.000000,10346.200195,10360.150391,10360.150391,173900,-1
2018-03-16,10345.150391,10346.299805,10180.250000,10195.150391,10195.150391,328500,-1
...,...,...,...,...,...,...,...
2022-10-06,17379.250000,17428.800781,17315.650391,17331.800781,17331.800781,265500,1
2022-10-07,17287.199219,17337.349609,17216.949219,17314.650391,17314.650391,216300,1
2022-10-10,17094.349609,17280.150391,17064.699219,17241.000000,17241.000000,234000,1


In [ ]:
data.to_csv('training_dataset.csv')
!cp data.csv "drive/My Drive/content/drive/MyDrive/StockPricePrediction/LSTMmodel/csv_files"